# Predictions - Try #2


- Author: Elmo Chavez
- Date: 02-Aug-2023

**Description**

> [...]


## Read the Data


### Libraries


In [1]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold, StratifiedKFold, StratifiedShuffleSplit
from sklearn.feature_selection import SelectKBest, mutual_info_classif, SelectPercentile, f_classif

import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import lightgbm as lgb

### Reading Data


In [2]:
current_directory = os.getcwd()  # Get the current working directory
parent_directory = os.path.dirname(current_directory)  # Get the parent directory

In [3]:
# Filenames
filename_list = ['2_tfr_multitaper_fp1.csv', '2_tfr_multitaper_all.csv', 
                 '2_tfr_morlet_fp1.csv', '2_tfr_morlet_all.csv', 
                 '2_psd_multitaper_fp1.csv', '2_psd_multitaper_all.csv',
                 '2_stats_fp1.csv', '2_stats_all.csv']

# Read all files
df_trainings = []
for file in filename_list:
    df_trainings.append(pd.read_csv(parent_directory+'/Training Datasets/'+file))

# Shape of the data
print('TFR Feature Extraction (Multitaper Method) - FP1:',df_trainings[0].shape)
print('TFR Feature Extraction (Multitaper Method) - All Channels:',df_trainings[1].shape)
print('TFR Feature Extraction (Morlet Method) - FP1:',df_trainings[2].shape)
print('TFR Feature Extraction (Morlet Method) - All Channels:',df_trainings[3].shape)
print('PSD Feature Extraction (Multitaper Method) - FP1:',df_trainings[4].shape)
print('PSD Feature Extraction (Multitaper Method) - All Channels:',df_trainings[5].shape)
print('Statistical Feature Extraction - FP1:',df_trainings[6].shape)
print('Statistical Feature Extraction - All Channels:',df_trainings[7].shape)

TFR Feature Extraction (Multitaper Method) - FP1: (51, 6)
TFR Feature Extraction (Multitaper Method) - All Channels: (51, 114)
TFR Feature Extraction (Morlet Method) - FP1: (51, 6)
TFR Feature Extraction (Morlet Method) - All Channels: (51, 114)
PSD Feature Extraction (Multitaper Method) - FP1: (51, 6)
PSD Feature Extraction (Multitaper Method) - All Channels: (51, 114)
Statistical Feature Extraction - FP1: (51, 6)
Statistical Feature Extraction - All Channels: (51, 114)


In [4]:
for df in df_trainings:
    print(df.columns.tolist())

['Fp1_mean', 'Fp1_standard_deviation', 'Fp1_variance', 'Fp1_peak_to_peak', 'Fp1_skewness', 'Fp1_kurtosis']
['Fp1_mean', 'Fp2_mean', 'F3_mean', 'F4_mean', 'C3_mean', 'C4_mean', 'P3_mean', 'P4_mean', 'O1_mean', 'O2_mean', 'F7_mean', 'F8_mean', 'T3_mean', 'T4_mean', 'T5_mean', 'T6_mean', 'Fz_mean', 'Cz_mean', 'Pz_mean', 'Fp1_standard_deviation', 'Fp2_standard_deviation', 'F3_standard_deviation', 'F4_standard_deviation', 'C3_standard_deviation', 'C4_standard_deviation', 'P3_standard_deviation', 'P4_standard_deviation', 'O1_standard_deviation', 'O2_standard_deviation', 'F7_standard_deviation', 'F8_standard_deviation', 'T3_standard_deviation', 'T4_standard_deviation', 'T5_standard_deviation', 'T6_standard_deviation', 'Fz_standard_deviation', 'Cz_standard_deviation', 'Pz_standard_deviation', 'Fp1_variance', 'Fp2_variance', 'F3_variance', 'F4_variance', 'C3_variance', 'C4_variance', 'P3_variance', 'P4_variance', 'O1_variance', 'O2_variance', 'F7_variance', 'F8_variance', 'T3_variance', 'T4_var

# Data Preprocessing


In [5]:
file_participants = 'participants_selected.csv'
df_participants = pd.read_csv(parent_directory+'/Training Datasets/'+file_participants)
df_participants.head()

,participant_id,Gender,Age,Group,MMSE
0,sub-002,1,78,0,22
1,sub-004,1,67,0,20
2,sub-005,0,70,0,22
3,sub-006,1,61,0,14
4,sub-007,1,79,0,20


In [6]:
cols_to_drop = ['participant_id','group']

for df in df_trainings:
    df['participant_id'] = df_participants['participant_id']
    df['gender'] = df_participants['Gender']
    df['age'] = df_participants['Age']
    df['MMSE'] = df_participants['MMSE']
    df['group'] = df_participants['Group']

In [7]:
features = []

TFR Multitaper FP1


In [8]:
features.append(df_trainings[0].drop(columns=cols_to_drop, axis=1).columns.tolist())
print(features[0])

X_tfr_multitaper_fp1 = df_trainings[0].drop(columns=cols_to_drop, axis=1).values
y_tfr_multitaper_fp1 = df_trainings[0]['group'].values

['Fp1_mean', 'Fp1_standard_deviation', 'Fp1_variance', 'Fp1_peak_to_peak', 'Fp1_skewness', 'Fp1_kurtosis', 'gender', 'age', 'MMSE']


TFR Multitaper All Channels


In [9]:
features.append(df_trainings[1].drop(columns=cols_to_drop, axis=1).columns.tolist())
print(features[1])

X_tfr_multitaper_all = df_trainings[1].drop(columns=cols_to_drop, axis=1).values
y_tfr_multitaper_all = df_trainings[1]['group'].values

['Fp1_mean', 'Fp2_mean', 'F3_mean', 'F4_mean', 'C3_mean', 'C4_mean', 'P3_mean', 'P4_mean', 'O1_mean', 'O2_mean', 'F7_mean', 'F8_mean', 'T3_mean', 'T4_mean', 'T5_mean', 'T6_mean', 'Fz_mean', 'Cz_mean', 'Pz_mean', 'Fp1_standard_deviation', 'Fp2_standard_deviation', 'F3_standard_deviation', 'F4_standard_deviation', 'C3_standard_deviation', 'C4_standard_deviation', 'P3_standard_deviation', 'P4_standard_deviation', 'O1_standard_deviation', 'O2_standard_deviation', 'F7_standard_deviation', 'F8_standard_deviation', 'T3_standard_deviation', 'T4_standard_deviation', 'T5_standard_deviation', 'T6_standard_deviation', 'Fz_standard_deviation', 'Cz_standard_deviation', 'Pz_standard_deviation', 'Fp1_variance', 'Fp2_variance', 'F3_variance', 'F4_variance', 'C3_variance', 'C4_variance', 'P3_variance', 'P4_variance', 'O1_variance', 'O2_variance', 'F7_variance', 'F8_variance', 'T3_variance', 'T4_variance', 'T5_variance', 'T6_variance', 'Fz_variance', 'Cz_variance', 'Pz_variance', 'Fp1_peak_to_peak', 'Fp2

TFR Morlet FP1


In [10]:
features.append(df_trainings[2].drop(columns=cols_to_drop, axis=1).columns.tolist())
print(features[2])

X_tfr_morlet_fp1 = df_trainings[2].drop(columns=cols_to_drop, axis=1).values
y_tfr_morlet_fp1 = df_trainings[2]['group'].values

['Fp1_mean', 'Fp1_standard_deviation', 'Fp1_variance', 'Fp1_peak_to_peak', 'Fp1_skewness', 'Fp1_kurtosis', 'gender', 'age', 'MMSE']


TFR Morlet All


In [11]:
features.append(df_trainings[3].drop(columns=cols_to_drop, axis=1).columns.tolist())
print(features[3])

X_tfr_morlet_all = df_trainings[3].drop(columns=cols_to_drop, axis=1).values
y_tfr_morlet_all = df_trainings[3]['group'].values

['Fp1_mean', 'Fp2_mean', 'F3_mean', 'F4_mean', 'C3_mean', 'C4_mean', 'P3_mean', 'P4_mean', 'O1_mean', 'O2_mean', 'F7_mean', 'F8_mean', 'T3_mean', 'T4_mean', 'T5_mean', 'T6_mean', 'Fz_mean', 'Cz_mean', 'Pz_mean', 'Fp1_standard_deviation', 'Fp2_standard_deviation', 'F3_standard_deviation', 'F4_standard_deviation', 'C3_standard_deviation', 'C4_standard_deviation', 'P3_standard_deviation', 'P4_standard_deviation', 'O1_standard_deviation', 'O2_standard_deviation', 'F7_standard_deviation', 'F8_standard_deviation', 'T3_standard_deviation', 'T4_standard_deviation', 'T5_standard_deviation', 'T6_standard_deviation', 'Fz_standard_deviation', 'Cz_standard_deviation', 'Pz_standard_deviation', 'Fp1_variance', 'Fp2_variance', 'F3_variance', 'F4_variance', 'C3_variance', 'C4_variance', 'P3_variance', 'P4_variance', 'O1_variance', 'O2_variance', 'F7_variance', 'F8_variance', 'T3_variance', 'T4_variance', 'T5_variance', 'T6_variance', 'Fz_variance', 'Cz_variance', 'Pz_variance', 'Fp1_peak_to_peak', 'Fp2

PSD Multitaper FP1


In [12]:
features.append(df_trainings[4].drop(columns=cols_to_drop, axis=1).columns.tolist())
print(features[4])

X_psd_multitaper_fp1 = df_trainings[4].drop(columns=cols_to_drop, axis=1).values
y_psd_multitaper_fp1 = df_trainings[4]['group'].values

['Fp1_mean', 'Fp1_standard_deviation', 'Fp1_variance', 'Fp1_peak_to_peak', 'Fp1_skewness', 'Fp1_kurtosis', 'gender', 'age', 'MMSE']


PSD Multitaper All Channels


In [13]:
features.append(df_trainings[5].drop(columns=cols_to_drop, axis=1).columns.tolist())
print(features[5])

X_psd_multitaper_all = df_trainings[5].drop(columns=cols_to_drop, axis=1).values
y_psd_multitaper_all = df_trainings[5]['group'].values

['Fp1_mean', 'Fp2_mean', 'F3_mean', 'F4_mean', 'C3_mean', 'C4_mean', 'P3_mean', 'P4_mean', 'O1_mean', 'O2_mean', 'F7_mean', 'F8_mean', 'T3_mean', 'T4_mean', 'T5_mean', 'T6_mean', 'Fz_mean', 'Cz_mean', 'Pz_mean', 'Fp1_standard_deviation', 'Fp2_standard_deviation', 'F3_standard_deviation', 'F4_standard_deviation', 'C3_standard_deviation', 'C4_standard_deviation', 'P3_standard_deviation', 'P4_standard_deviation', 'O1_standard_deviation', 'O2_standard_deviation', 'F7_standard_deviation', 'F8_standard_deviation', 'T3_standard_deviation', 'T4_standard_deviation', 'T5_standard_deviation', 'T6_standard_deviation', 'Fz_standard_deviation', 'Cz_standard_deviation', 'Pz_standard_deviation', 'Fp1_variance', 'Fp2_variance', 'F3_variance', 'F4_variance', 'C3_variance', 'C4_variance', 'P3_variance', 'P4_variance', 'O1_variance', 'O2_variance', 'F7_variance', 'F8_variance', 'T3_variance', 'T4_variance', 'T5_variance', 'T6_variance', 'Fz_variance', 'Cz_variance', 'Pz_variance', 'Fp1_peak_to_peak', 'Fp2

Simple Stats FP1


In [14]:
features.append(df_trainings[6].drop(columns=cols_to_drop, axis=1).columns.tolist())
print(features[6])

X_stats_fp1 = df_trainings[6].drop(columns=cols_to_drop, axis=1).values
y_stats_fp1 = df_trainings[6]['group'].values

['Fp1_mean', 'Fp1_standard_deviation', 'Fp1_variance', 'Fp1_peak_to_peak', 'Fp1_skewness', 'Fp1_kurtosis', 'gender', 'age', 'MMSE']


Simple Stats All Channels


In [15]:
features.append(df_trainings[7].drop(columns=cols_to_drop, axis=1).columns.tolist())
print(features[7])

X_stats_all = df_trainings[7].drop(columns=cols_to_drop, axis=1).values
y_stats_all = df_trainings[7]['group'].values

['Fp1_mean', 'Fp2_mean', 'F3_mean', 'F4_mean', 'C3_mean', 'C4_mean', 'P3_mean', 'P4_mean', 'O1_mean', 'O2_mean', 'F7_mean', 'F8_mean', 'T3_mean', 'T4_mean', 'T5_mean', 'T6_mean', 'Fz_mean', 'Cz_mean', 'Pz_mean', 'Fp1_standard_deviation', 'Fp2_standard_deviation', 'F3_standard_deviation', 'F4_standard_deviation', 'C3_standard_deviation', 'C4_standard_deviation', 'P3_standard_deviation', 'P4_standard_deviation', 'O1_standard_deviation', 'O2_standard_deviation', 'F7_standard_deviation', 'F8_standard_deviation', 'T3_standard_deviation', 'T4_standard_deviation', 'T5_standard_deviation', 'T6_standard_deviation', 'Fz_standard_deviation', 'Cz_standard_deviation', 'Pz_standard_deviation', 'Fp1_variance', 'Fp2_variance', 'F3_variance', 'F4_variance', 'C3_variance', 'C4_variance', 'P3_variance', 'P4_variance', 'O1_variance', 'O2_variance', 'F7_variance', 'F8_variance', 'T3_variance', 'T4_variance', 'T5_variance', 'T6_variance', 'Fz_variance', 'Cz_variance', 'Pz_variance', 'Fp1_peak_to_peak', 'Fp2

# Predictions


Functions required to Predict and Save the Results


In [16]:
#cols_results = ['Feature Extraction','Frequencies','Channels','Metrics','Cross Validate Method','Feature Selection','Metrics_Selected','ML Model','Accuracy']
#df_results = pd.DataFrame(columns=cols_results)

filename = '2_results.csv'
df_results = pd.read_csv(current_directory+'/'+filename)

def append_result(result):      
    index = len(df_results)
    df_results.loc[index] = result
    
def remove_items(df, approach, frequencies, channels):
    mask = (df['Feature Extraction'] != approach) | (df['Frequencies'] != frequencies) | (df['Channels'] != channels)
    filtered_df = df[mask]
    return filtered_df

def save_results(df, path, filename):
    df.to_csv(path+'/'+filename, index=False)  
    print('Results saved, records:',len(df))
    
    
print('Results: ', len(df_results))

def predictor(X, y, approach, frequencies, channels, features_names, selector_k, selector_perc):
    # Define the classifiers
    svm_classifier = SVC()
    random_forest_classifier = RandomForestClassifier()
    adaboost_classifier = AdaBoostClassifier()
    xgboost_classifier = xgb.XGBClassifier()
    lightgbm_classifier = lgb.LGBMClassifier()

    classifiers = [svm_classifier, random_forest_classifier, adaboost_classifier, 
                   xgboost_classifier, lightgbm_classifier]
    classifier_names = ['SVM', 'Random Forest', 'AdaBoost', 
                        'XGBoost', 'LightGBM']

    # Create a cross-validator with 5 folds
    n_splits = 5
    test_size = 0.2

    cv_method = [KFold(n_splits=n_splits, shuffle=True, random_state=42),
                StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42),
                StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=42)]
    cv_method_names = ['KFold','StratifiedKFold','StratifiedShuffleSplit']

    # Create the Feature Selector method
    feat_sel_method =   [SelectKBest(score_func=f_classif, k=selector_k), SelectKBest(score_func=mutual_info_classif, k=selector_k),
                        SelectPercentile(score_func=f_classif, percentile=selector_perc), SelectPercentile(score_func=mutual_info_classif, percentile=selector_perc)]
    feat_sel_method_names = ['SelectKBest - f_classif','SelectKBest - mutual_info_classif',
                            'SelectPercentile - f_classif', 'SelectPercentile - mutual_info_classif']

    for clf, clf_name in zip(classifiers, classifier_names):
        print('Running',clf_name)
        for cv, cv_name in zip(cv_method, cv_method_names):
            for selector, sel_name in zip(feat_sel_method, feat_sel_method_names):
                scores = []
                for train_index, test_index in cv.split(X, y):
                    # Split the data into training and testing sets based on the current fold indices
                    X_train, X_test = X[train_index], X[test_index]
                    y_train, y_test = y[train_index], y[test_index]
                    
                    # Feature Selection
                    X_train_selected = selector.fit_transform(X_train, y_train)
                    selected_index = selector.get_support()
                    X_test_selected = selector.transform(X_test)
                    
                    # Train the classifier on the training data
                    clf.fit(X_train_selected, y_train)
                    
                    # Make predictions on the test data
                    y_pred = clf.predict(X_test_selected)
                    
                    # Calculate accuracy for this fold and append to the scores list
                    accuracy = np.mean(y_pred == y_test)
                    scores.append(accuracy)
        
                #save results
                result = [approach, frequencies, channels, features_names, cv_name, sel_name, 
                          [features_names[i] for i, mask in enumerate(selected_index) if mask], clf_name, np.mean(scores)]
                append_result(result)

df_results.head()

Results:  360


,Feature Extraction,Frequencies,Channels,Metrics,Cross Validate Method,Feature Selection,Metrics_Selected,ML Model,Accuracy
0,TFR Multitaper,1-45 Hz,FP1,"['Fp1_mean', 'Fp1_standard_deviation', 'Fp1_va...",KFold,SelectKBest - f_classif,"['Fp1_peak_to_peak', 'Fp1_skewness', 'Fp1_kurt...",SVM,0.645455
1,TFR Multitaper,1-45 Hz,FP1,"['Fp1_mean', 'Fp1_standard_deviation', 'Fp1_va...",KFold,SelectKBest - mutual_info_classif,"['Fp1_peak_to_peak', 'Fp1_skewness', 'Fp1_kurt...",SVM,0.645455
2,TFR Multitaper,1-45 Hz,FP1,"['Fp1_mean', 'Fp1_standard_deviation', 'Fp1_va...",KFold,SelectPercentile - f_classif,"['gender', 'age', 'MMSE']",SVM,0.645455
3,TFR Multitaper,1-45 Hz,FP1,"['Fp1_mean', 'Fp1_standard_deviation', 'Fp1_va...",KFold,SelectPercentile - mutual_info_classif,"['Fp1_mean', 'age', 'MMSE']",SVM,0.645455
4,TFR Multitaper,1-45 Hz,FP1,"['Fp1_mean', 'Fp1_standard_deviation', 'Fp1_va...",StratifiedKFold,SelectKBest - f_classif,"['Fp1_peak_to_peak', 'Fp1_skewness', 'Fp1_kurt...",SVM,0.647273


## TFR Multitaper: Channel FP1


In [18]:
# Variables for Results
approach = 'TFR Multitaper'
frequencies = '1-45 Hz'
channels = 'FP1'

print('Number of features:', len(features[0]))

# Clean df_results
df_results = remove_items(df_results, approach, frequencies, channels)
#Prediction
predictor(X_tfr_multitaper_fp1, y_tfr_multitaper_fp1,approach,frequencies,channels,features[0], 6, 35)
# Save results
save_results(df_results,current_directory,filename)
# Show new results
df_results[(df_results['Feature Extraction'] == approach) &  (df_results['Frequencies'] == frequencies) &  (df_results['Channels'] == channels)].sort_values('Accuracy', ascending=False)

Number of features: 9
Results saved, records: 60


,Feature Extraction,Frequencies,Channels,Metrics,Cross Validate Method,Feature Selection,Metrics_Selected,ML Model,Accuracy
35,TFR Multitaper,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedShuffleSplit,SelectPercentile - mutual_info_classif,"[Fp1_mean, age, MMSE]",AdaBoost,0.654545
22,TFR Multitaper,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedShuffleSplit,SelectPercentile - f_classif,"[gender, age, MMSE]",Random Forest,0.654545
23,TFR Multitaper,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedShuffleSplit,SelectPercentile - mutual_info_classif,"[gender, age, MMSE]",Random Forest,0.654545
7,TFR Multitaper,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedKFold,SelectPercentile - mutual_info_classif,"[gender, age, MMSE]",SVM,0.647273
52,TFR Multitaper,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedKFold,SelectKBest - f_classif,"[Fp1_peak_to_peak, Fp1_skewness, Fp1_kurtosis,...",LightGBM,0.647273
53,TFR Multitaper,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedKFold,SelectKBest - mutual_info_classif,"[Fp1_peak_to_peak, Fp1_skewness, Fp1_kurtosis,...",LightGBM,0.647273
54,TFR Multitaper,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedKFold,SelectPercentile - f_classif,"[gender, age, MMSE]",LightGBM,0.647273
55,TFR Multitaper,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedKFold,SelectPercentile - mutual_info_classif,"[Fp1_mean, Fp1_standard_deviation, MMSE]",LightGBM,0.647273
6,TFR Multitaper,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedKFold,SelectPercentile - f_classif,"[gender, age, MMSE]",SVM,0.647273
5,TFR Multitaper,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedKFold,SelectKBest - mutual_info_classif,"[Fp1_peak_to_peak, Fp1_skewness, Fp1_kurtosis,...",SVM,0.647273


## TFR Multitaper: All Channels


In [19]:
# Variables for Results
approach = 'TFR Multitaper'
frequencies = '1-45 Hz'
channels = 'All'

print('Number of features:', len(features[1]))

# Clean df_results
df_results = remove_items(df_results, approach, frequencies, channels)
#Prediction
predictor(X_tfr_multitaper_all, y_tfr_multitaper_all,approach,frequencies,channels,features[1], 30, 30)
# Save results
save_results(df_results,current_directory,filename)
# Show new results
df_results[(df_results['Feature Extraction'] == approach) &  (df_results['Frequencies'] == frequencies) &  (df_results['Channels'] == channels)].sort_values('Accuracy', ascending=False)

Number of features: 117


,Feature Extraction,Frequencies,Channels,Metrics,Cross Validate Method,Feature Selection,Metrics_Selected,ML Model,Accuracy
83,TFR Multitaper,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedShuffleSplit,SelectPercentile - mutual_info_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",Random Forest,0.672727
74,TFR Multitaper,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",KFold,SelectPercentile - f_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",Random Forest,0.665455
75,TFR Multitaper,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",KFold,SelectPercentile - mutual_info_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",Random Forest,0.663636
80,TFR Multitaper,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedShuffleSplit,SelectKBest - f_classif,"[F8_skewness, T3_skewness, T4_skewness, T5_ske...",Random Forest,0.654545
91,TFR Multitaper,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedKFold,SelectPercentile - mutual_info_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",AdaBoost,0.649091
115,TFR Multitaper,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedKFold,SelectPercentile - mutual_info_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",LightGBM,0.647273
64,TFR Multitaper,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedKFold,SelectKBest - f_classif,"[F8_skewness, T3_skewness, T4_skewness, T5_ske...",SVM,0.647273
65,TFR Multitaper,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedKFold,SelectKBest - mutual_info_classif,"[F8_skewness, T3_skewness, T4_skewness, T5_ske...",SVM,0.647273
66,TFR Multitaper,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedKFold,SelectPercentile - f_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",SVM,0.647273
67,TFR Multitaper,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedKFold,SelectPercentile - mutual_info_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",SVM,0.647273


## TFR Morlet: FP1 Channel


In [21]:
# Variables for Results
approach = 'TFR Morlet'
frequencies = '1-45 Hz'
channels = 'FP1'

print('Number of features:', len(features[2]))

# Clean df_results
df_results = remove_items(df_results, approach, frequencies, channels)
#Prediction
predictor(X_tfr_morlet_fp1, y_tfr_morlet_fp1, approach, frequencies, channels, features[2], 5, 35)
# Save results
save_results(df_results,current_directory,filename)
# Show new results
df_results[(df_results['Feature Extraction'] == approach) &  (df_results['Frequencies'] == frequencies) &  (df_results['Channels'] == channels)].sort_values('Accuracy', ascending=False)

Number of features: 9
Results saved, records: 180


,Feature Extraction,Frequencies,Channels,Metrics,Cross Validate Method,Feature Selection,Metrics_Selected,ML Model,Accuracy
167,TFR Morlet,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedShuffleSplit,SelectPercentile - mutual_info_classif,"[Fp1_mean, age, MMSE]",XGBoost,0.727273
153,TFR Morlet,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedShuffleSplit,SelectKBest - mutual_info_classif,"[Fp1_skewness, Fp1_kurtosis, gender, age, MMSE]",AdaBoost,0.709091
133,TFR Morlet,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",KFold,SelectKBest - mutual_info_classif,"[Fp1_variance, Fp1_peak_to_peak, Fp1_skewness,...",Random Forest,0.663636
147,TFR Morlet,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",KFold,SelectPercentile - mutual_info_classif,"[Fp1_mean, Fp1_standard_deviation, MMSE]",AdaBoost,0.661818
155,TFR Morlet,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedShuffleSplit,SelectPercentile - mutual_info_classif,"[gender, age, MMSE]",AdaBoost,0.654545
140,TFR Morlet,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedShuffleSplit,SelectKBest - f_classif,"[Fp1_skewness, Fp1_kurtosis, gender, age, MMSE]",Random Forest,0.654545
175,TFR Morlet,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedKFold,SelectPercentile - mutual_info_classif,"[Fp1_mean, Fp1_standard_deviation, MMSE]",LightGBM,0.647273
172,TFR Morlet,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedKFold,SelectKBest - f_classif,"[Fp1_skewness, Fp1_kurtosis, gender, age, MMSE]",LightGBM,0.647273
174,TFR Morlet,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedKFold,SelectPercentile - f_classif,"[gender, age, MMSE]",LightGBM,0.647273
173,TFR Morlet,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedKFold,SelectKBest - mutual_info_classif,"[Fp1_standard_deviation, Fp1_variance, Fp1_pea...",LightGBM,0.647273


## TFR Morlet: All Channels


In [17]:
# Variables for Results
approach = 'TFR Morlet'
frequencies = '1-45 Hz'
channels = 'All'

print('Number of features:', len(features[3]))

# Clean df_results
df_results = remove_items(df_results, approach, frequencies, channels)
#Prediction
predictor(X_tfr_morlet_all, y_tfr_morlet_all, approach, frequencies, channels, features[3], 30, 30)
# Save results
save_results(df_results,current_directory,filename)
# Show new results
df_results[(df_results['Feature Extraction'] == approach) &  (df_results['Frequencies'] == frequencies) &  (df_results['Channels'] == channels)].sort_values('Accuracy', ascending=False)

Number of features: 117
Running SVM
Running Random Forest
Running AdaBoost
Running XGBoost
Running LightGBM
Results saved, records: 240


,Feature Extraction,Frequencies,Channels,Metrics,Cross Validate Method,Feature Selection,Metrics_Selected,ML Model,Accuracy
213,TFR Morlet,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedShuffleSplit,SelectKBest - mutual_info_classif,"[F8_skewness, T3_skewness, T4_skewness, T5_ske...",AdaBoost,0.709091
227,TFR Morlet,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedShuffleSplit,SelectPercentile - mutual_info_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",XGBoost,0.709091
201,TFR Morlet,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedShuffleSplit,SelectKBest - mutual_info_classif,"[O2_skewness, F7_skewness, F8_skewness, T3_ske...",Random Forest,0.672727
217,TFR Morlet,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",KFold,SelectKBest - mutual_info_classif,"[O2_skewness, F7_skewness, F8_skewness, T3_ske...",XGBoost,0.665455
193,TFR Morlet,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",KFold,SelectKBest - mutual_info_classif,"[F7_skewness, F8_skewness, T3_skewness, T4_ske...",Random Forest,0.663636
205,TFR Morlet,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",KFold,SelectKBest - mutual_info_classif,"[F7_skewness, F8_skewness, T3_skewness, T4_ske...",AdaBoost,0.661818
195,TFR Morlet,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",KFold,SelectPercentile - mutual_info_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",Random Forest,0.661818
215,TFR Morlet,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedShuffleSplit,SelectPercentile - mutual_info_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",AdaBoost,0.654545
200,TFR Morlet,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedShuffleSplit,SelectKBest - f_classif,"[F8_skewness, T3_skewness, T4_skewness, T5_ske...",Random Forest,0.654545
185,TFR Morlet,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedKFold,SelectKBest - mutual_info_classif,"[O2_skewness, F7_skewness, F8_skewness, T3_ske...",SVM,0.647273


## PSD Multitaper: FP1 Channel


In [18]:
# Variables for Results
approach = 'PSD Multitaper'
frequencies = '1-45 Hz'
channels = 'FP1'

print('Number of features:', len(features[4]))

# Clean df_results
df_results = remove_items(df_results, approach, frequencies, channels)
#Prediction
predictor(X_psd_multitaper_fp1, y_psd_multitaper_fp1, approach, frequencies, channels, features[4], 6, 30)
# Save results
save_results(df_results,current_directory,filename)
# Show new results
df_results[(df_results['Feature Extraction'] == approach) &  (df_results['Frequencies'] == frequencies) &  (df_results['Channels'] == channels)].sort_values('Accuracy', ascending=False)

Number of features: 9
Running SVM
Running Random Forest
Running AdaBoost
Running XGBoost
Running LightGBM
Results saved, records: 300


,Feature Extraction,Frequencies,Channels,Metrics,Cross Validate Method,Feature Selection,Metrics_Selected,ML Model,Accuracy
287,PSD Multitaper,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedShuffleSplit,SelectPercentile - mutual_info_classif,"[age, MMSE]",XGBoost,0.672727
252,PSD Multitaper,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",KFold,SelectKBest - f_classif,"[Fp1_peak_to_peak, Fp1_skewness, Fp1_kurtosis,...",Random Forest,0.663636
267,PSD Multitaper,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",KFold,SelectPercentile - mutual_info_classif,"[gender, MMSE]",AdaBoost,0.663636
275,PSD Multitaper,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedShuffleSplit,SelectPercentile - mutual_info_classif,"[age, MMSE]",AdaBoost,0.654545
271,PSD Multitaper,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedKFold,SelectPercentile - mutual_info_classif,"[Fp1_mean, MMSE]",AdaBoost,0.649091
293,PSD Multitaper,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedKFold,SelectKBest - mutual_info_classif,"[Fp1_peak_to_peak, Fp1_skewness, Fp1_kurtosis,...",LightGBM,0.647273
245,PSD Multitaper,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedKFold,SelectKBest - mutual_info_classif,"[Fp1_peak_to_peak, Fp1_skewness, Fp1_kurtosis,...",SVM,0.647273
246,PSD Multitaper,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedKFold,SelectPercentile - f_classif,"[gender, age, MMSE]",SVM,0.647273
247,PSD Multitaper,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedKFold,SelectPercentile - mutual_info_classif,"[Fp1_mean, MMSE]",SVM,0.647273
295,PSD Multitaper,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedKFold,SelectPercentile - mutual_info_classif,"[Fp1_mean, MMSE]",LightGBM,0.647273


## PSD Multitaper: All Channel


In [17]:
# Variables for Results
approach = 'PSD Multitaper'
frequencies = '1-45 Hz'
channels = 'All'

print('Number of features:', len(features[5]))

# Clean df_results
df_results = remove_items(df_results, approach, frequencies, channels)
#Prediction
predictor(X_psd_multitaper_all, y_psd_multitaper_all, approach, frequencies, channels, features[5], 35, 35)
# Save results
save_results(df_results,current_directory,filename)
# Show new results
df_results[(df_results['Feature Extraction'] == approach) &  (df_results['Frequencies'] == frequencies) &  (df_results['Channels'] == channels)].sort_values('Accuracy', ascending=False)

Number of features: 117
Running SVM
Running Random Forest
Running AdaBoost
Running XGBoost
Running LightGBM
Results saved, records: 360


,Feature Extraction,Frequencies,Channels,Metrics,Cross Validate Method,Feature Selection,Metrics_Selected,ML Model,Accuracy
347,PSD Multitaper,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedShuffleSplit,SelectPercentile - mutual_info_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",XGBoost,0.672727
335,PSD Multitaper,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedShuffleSplit,SelectPercentile - mutual_info_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",AdaBoost,0.654545
304,PSD Multitaper,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedKFold,SelectKBest - f_classif,"[P3_skewness, P4_skewness, O1_skewness, O2_ske...",SVM,0.647273
305,PSD Multitaper,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedKFold,SelectKBest - mutual_info_classif,"[P3_skewness, P4_skewness, O1_skewness, O2_ske...",SVM,0.647273
306,PSD Multitaper,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedKFold,SelectPercentile - f_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",SVM,0.647273
307,PSD Multitaper,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedKFold,SelectPercentile - mutual_info_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",SVM,0.647273
355,PSD Multitaper,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedKFold,SelectPercentile - mutual_info_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",LightGBM,0.647273
354,PSD Multitaper,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedKFold,SelectPercentile - f_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",LightGBM,0.647273
353,PSD Multitaper,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedKFold,SelectKBest - mutual_info_classif,"[P3_skewness, P4_skewness, O1_skewness, O2_ske...",LightGBM,0.647273
352,PSD Multitaper,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedKFold,SelectKBest - f_classif,"[P3_skewness, P4_skewness, O1_skewness, O2_ske...",LightGBM,0.647273


## Stats: FP1 Channel


In [20]:
# Variables for Results
approach = 'Statistics'
frequencies = '1-45 Hz'
channels = 'FP1'

print('Number of features:', len(features[6]))

# Clean df_results
df_results = remove_items(df_results, approach, frequencies, channels)
#Prediction
predictor(X_stats_fp1, y_stats_fp1, approach, frequencies, channels, features[6], 5, 35)
# Show new results
df_results[(df_results['Feature Extraction'] == approach) &  (df_results['Frequencies'] == frequencies) &  (df_results['Channels'] == channels)].sort_values('Accuracy', ascending=False)

Number of features: 9


,Feature Extraction,Frequencies,Channels,Metrics,Cross Validate Method,Feature Selection,Metrics_Selected,ML Model,Accuracy
225,Statistics,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedShuffleSplit,SelectKBest - mutual_info_classif,"[Fp1_peak_to_peak, Fp1_skewness, Fp1_kurtosis,...",XGBoost,0.709091
214,Statistics,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedShuffleSplit,SelectPercentile - f_classif,"[gender, age, MMSE]",AdaBoost,0.690909
212,Statistics,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedShuffleSplit,SelectKBest - f_classif,"[Fp1_skewness, Fp1_kurtosis, gender, age, MMSE]",AdaBoost,0.690909
220,Statistics,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedKFold,SelectKBest - f_classif,"[Fp1_skewness, Fp1_kurtosis, gender, age, MMSE]",XGBoost,0.689091
221,Statistics,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedKFold,SelectKBest - mutual_info_classif,"[Fp1_skewness, Fp1_kurtosis, gender, age, MMSE]",XGBoost,0.689091
200,Statistics,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedShuffleSplit,SelectKBest - f_classif,"[Fp1_skewness, Fp1_kurtosis, gender, age, MMSE]",Random Forest,0.672727
213,Statistics,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedShuffleSplit,SelectKBest - mutual_info_classif,"[Fp1_skewness, Fp1_kurtosis, gender, age, MMSE]",AdaBoost,0.672727
215,Statistics,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedShuffleSplit,SelectPercentile - mutual_info_classif,"[gender, age, MMSE]",AdaBoost,0.672727
202,Statistics,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedShuffleSplit,SelectPercentile - f_classif,"[gender, age, MMSE]",Random Forest,0.654545
224,Statistics,1-45 Hz,FP1,"[Fp1_mean, Fp1_standard_deviation, Fp1_varianc...",StratifiedShuffleSplit,SelectKBest - f_classif,"[Fp1_skewness, Fp1_kurtosis, gender, age, MMSE]",XGBoost,0.654545


## Stats: All Channels


In [17]:
# Variables for Results
approach = 'Statistics'
frequencies = '1-45 Hz'
channels = 'All'

print('Number of features:', len(features[7]))

# Clean df_results
df_results = remove_items(df_results, approach, frequencies, channels)
#Prediction
predictor(X_stats_all, y_stats_all, approach, frequencies, channels, features[7], 30, 30)
# Save results
save_results(df_results,current_directory,filename)
# Show new results
df_results[(df_results['Feature Extraction'] == approach) &  (df_results['Frequencies'] == frequencies) &  (df_results['Channels'] == channels)].sort_values('Accuracy', ascending=False)

Number of features: 117
Running SVM
Running Random Forest
Running AdaBoost
Running XGBoost
Running LightGBM
Results saved, records: 420


,Feature Extraction,Frequencies,Channels,Metrics,Cross Validate Method,Feature Selection,Metrics_Selected,ML Model,Accuracy
375,Statistics,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",KFold,SelectPercentile - mutual_info_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",Random Forest,0.721818
405,Statistics,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedShuffleSplit,SelectKBest - mutual_info_classif,"[F7_skewness, F8_skewness, T3_skewness, T4_ske...",XGBoost,0.709091
394,Statistics,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedShuffleSplit,SelectPercentile - f_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",AdaBoost,0.690909
392,Statistics,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedShuffleSplit,SelectKBest - f_classif,"[F8_skewness, T3_skewness, T4_skewness, T5_ske...",AdaBoost,0.690909
395,Statistics,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedShuffleSplit,SelectPercentile - mutual_info_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",AdaBoost,0.690909
378,Statistics,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedKFold,SelectPercentile - f_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",Random Forest,0.689091
402,Statistics,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedKFold,SelectPercentile - f_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",XGBoost,0.689091
401,Statistics,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedKFold,SelectKBest - mutual_info_classif,"[F8_skewness, T3_skewness, T4_skewness, T5_ske...",XGBoost,0.689091
400,Statistics,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedKFold,SelectKBest - f_classif,"[F8_skewness, T3_skewness, T4_skewness, T5_ske...",XGBoost,0.689091
376,Statistics,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedKFold,SelectKBest - f_classif,"[F8_skewness, T3_skewness, T4_skewness, T5_ske...",Random Forest,0.689091


# Results


In [18]:
df_results.sort_values('Accuracy',ascending=False).head(20)

,Feature Extraction,Frequencies,Channels,Metrics,Cross Validate Method,Feature Selection,Metrics_Selected,ML Model,Accuracy
167,TFR Morlet,1-45 Hz,FP1,"['Fp1_mean', 'Fp1_standard_deviation', 'Fp1_va...",StratifiedShuffleSplit,SelectPercentile - mutual_info_classif,"['Fp1_mean', 'age', 'MMSE']",XGBoost,0.727273
375,Statistics,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",KFold,SelectPercentile - mutual_info_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",Random Forest,0.721818
213,TFR Morlet,1-45 Hz,All,"['Fp1_mean', 'Fp2_mean', 'F3_mean', 'F4_mean',...",StratifiedShuffleSplit,SelectKBest - mutual_info_classif,"['F8_skewness', 'T3_skewness', 'T4_skewness', ...",AdaBoost,0.709091
153,TFR Morlet,1-45 Hz,FP1,"['Fp1_mean', 'Fp1_standard_deviation', 'Fp1_va...",StratifiedShuffleSplit,SelectKBest - mutual_info_classif,"['Fp1_skewness', 'Fp1_kurtosis', 'gender', 'ag...",AdaBoost,0.709091
227,TFR Morlet,1-45 Hz,All,"['Fp1_mean', 'Fp2_mean', 'F3_mean', 'F4_mean',...",StratifiedShuffleSplit,SelectPercentile - mutual_info_classif,"['Fp1_mean', 'Fp2_mean', 'F3_mean', 'F4_mean',...",XGBoost,0.709091
405,Statistics,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedShuffleSplit,SelectKBest - mutual_info_classif,"[F7_skewness, F8_skewness, T3_skewness, T4_ske...",XGBoost,0.709091
392,Statistics,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedShuffleSplit,SelectKBest - f_classif,"[F8_skewness, T3_skewness, T4_skewness, T5_ske...",AdaBoost,0.690909
394,Statistics,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedShuffleSplit,SelectPercentile - f_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",AdaBoost,0.690909
395,Statistics,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedShuffleSplit,SelectPercentile - mutual_info_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",AdaBoost,0.690909
378,Statistics,1-45 Hz,All,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",StratifiedKFold,SelectPercentile - f_classif,"[Fp1_mean, Fp2_mean, F3_mean, F4_mean, C3_mean...",Random Forest,0.689091


**Discussion**

Overall, all the Feature Extractions made have similar performance, while some of them may vary depending on the Feature Selection methods or the cross-validation methods, The final accuracy for each case is not enough to determine a good conclusion.
